# Homework for Session 3 - Introduction to LangChain
## Working with LangChain Models, Prompts and Chains
**Due 14.11.2023**

This Jupyter notebook contains a demo application and additional tasks related to the demo application. You should read the tasks at the bottom of this notebook first and then work through this notebook. To use this notebook, you need an API key from an LLM provider. The two implemented options are OpenAI's *gpt-3.5-turbo* model, which costs a bit of money but it shouldn't cost more than a few cents to work through this notebook, and Coheres chat model, which can be used for free via a free trial API key.

You can get your API keys after registering here: \
[OpenAI API key](https://platform.openai.com/api-keys) \
[Cohere API key](https://dashboard.cohere.com/api-keys)

### Installing LangChain and OpenAI/Cohere

For this notebook we will install specific versions of the libraries since OpenAI released version 1.0.0 on 05.11.2023 and this version had changes that didn't work with the newest version of LangChain. If you run into troubles feel free to reach out via Mattermost.

In [ ]:
!pip install langchain==0.0.331
!pip install openai==0.28.1
!pip install cohere

### Example application

In [ ]:
from langchain.chat_models import ChatOpenAI, ChatCohere
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
import getpass

#### Choosing the LLM

In this example we provide two options, but you can try and replace it with any other supported chat LLM if you want to.

[LangChain supported chat models](https://python.langchain.com/docs/integrations/chat/)

In [ ]:
OPENAI_API_KEY = getpass.getpass("Enter your OpenAI API key: ")
# COHERE_API_KEY = getpass.getpass("Enter your Cohere API key: ")

In [ ]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5, openai_api_key=OPENAI_API_KEY)
# llm = ChatCohere(temperature=0.5, cohere_api_key=COHERE_API_KEY)

In [ ]:
classification_prompt = ChatPromptTemplate.from_template(
    """
    You will be provided with an e-mail received by a recipient. \
    The e-mail will be delimited by triple backticks. \
    Classify the e-mail into one of the following e-mail categories \
    and answer only with the category it belongs to. \

    Categories:
    1. Newsletter
    2. Business
    3. Order Confirmation

    Examples:
    E-Mail: ```E-Mail content```
    Response: Newsletter

    E-Mail: ```E-Mail content```
    Response: Business

    E-Mail: ```E-Mail content```
    Response: Order Confirmation

    E-Mail: ```{email}```
    """
)

classification_chain = LLMChain(
    llm=llm,
    prompt=classification_prompt,
    output_key="mailtype"
)

In [ ]:
summarization_prompt = ChatPromptTemplate.from_template(
    """
    Your task is to generate a short summary of a {mailtype} e-mail \
    to inform the recipient of the e-mail about the most important \
    information within the e-mail.

    Summarize the e-amil below, delimited by triple
    backticks, in at most 50 words focusing on the most important aspects \
    of the e-mail.

    E-Mail: ```{email}```
    """
)

summarization_chain = LLMChain(
    llm=llm,
    prompt=summarization_prompt,
    output_key="summary"
)

In [ ]:
email_answer_prompt = ChatPromptTemplate.from_template(
    """
      If the type of the email is 'Business', please draft a response to the email provided below in triple backticks. Otherwise, kindly provide a brief explanation as to why an answer email draft cannot be provided.

      E-Mail Type: {mailtype}
      E-Mail: ```{email}```
    """
)

email_answer_chain = LLMChain(
    llm=llm,
    prompt=email_answer_prompt,
    output_key="email_answer"
)

In [ ]:
main_chain = SequentialChain(
    chains=[classification_chain, summarization_chain, email_answer_chain],
    input_variables=["email"],
    output_variables=["summary", "email_answer", "mailtype"],
    verbose=True
)

### Example inputs

#### Example 1

In [ ]:
example_1 = """
Dear Mr. Smith,

I hope this email finds you well. I'm Emily Parker from XYZ Solutions. I came across Acme Innovations and was impressed by your recent market expansion. I believe there's potential for us to collaborate on developing a new software solution. Would you be available for a brief call next week to discuss this further?

Looking forward to your response.

Best regards,

Emily Parker
Business Development Manager
XYZ Solutions
Email: emily.parker@xyzsolutions.com
Phone: +1-555-123-9876
"""

In [ ]:
result = main_chain(example_1)

In [ ]:
for key in result.keys():
  print(f"{key}:")
  print(result[key])
  print("\n\n")

#### Example 2

In [ ]:
example_2 = """
Hi there,

Welcome to TechVibe Innovations' monthly scoop! Here's a glimpse of what we've got:

1. Unveiling Our Stellar Phoenix Series
The newest in tech—meet the game-changer enhancing your experience.

2. AI in Healthcare: The Next Frontier
Explore how AI is shaping the future of healthcare in our exclusive insight.

3. Meet Our Ethical AI Expert
Dr. Jane Doe talks about AI ethics in an unmissable interview.

Get ahead with our blog! Subscribe for deep dives, tips, and more.

We want your thoughts! Share your ideas for future newsletters.

Thanks for being part of our innovation journey!

Best,
TechVibe Innovations Team
  """

In [ ]:
result = main_chain(example_2)

In [ ]:
for key in result.keys():
  print(f"{key}:")
  print(result[key])
  print("\n\n")

#### Example 3

In [ ]:
example_3 = """
Dear Customer,

Thank you for your recent order with SuperGadgets Inc.

Here are the details:

Order Number: #56789
Order Date: November 6, 2023
Delivery Address: 1234 Elm Street, Anytown, USA

Items Ordered:

1. Blue Backpack x 2
2. Wireless Earbuds x 1

Total Amount: $89.99 (incl. shipping & tax)

Your order is being processed and will be shipped soon. You'll receive tracking details once dispatched.

For any questions, reach out to us at support@supergadgets.com or call 800-123-4567. View our return policy here.

Best Regards,
SuperGadgets Inc. Team
"""

In [ ]:
result = main_chain(example_3)

In [ ]:
for key in result.keys():
  print(f"{key}:")
  print(result[key])
  print("\n\n")

### Tasks:
1. Create a simple sketch showing the flow of the application, the inputs and outputs of the components of the application, and briefly explain in 1-2 sentences what the application does.

2. Analyze the code and answer the following questions with a brief explanation:
- Does the application produce good results according to the use case?
- What happens if you change the order of the chains in the SequentialChain?
- What prompting techniques do you recognize in the individual prompts of the used chains
- Create a fourth example (e.g. with ChatGPT) for a type that wasn't defined in the prompt. What happens if the type of email doesn't fall into the defined categories?

3. Make the following changes to the code:
- If some chains don't give meaningful results for some examples, try changing the prompt so that it works. Also check that the other examples still work.
- Change the *classification_prompt* to explicitly handle unknown email types.
- Add another email type category of your choice to the *classification_prompt* that should also be answered through the *email_answer_chain*.
- Add another chain that takes the summary and generates a meaningful headline of up to 10 words.

4. Get creative and create your own SequentialChain with at least 3 chains.